In [2]:
# SVM
# Support vector machines (SVMs) are a set of supervised learning methods used for 
# classification, regression and outliers detection.

# The advantages of support vector machines are:
    # Effective in high dimensional spaces.
    # Still effective in cases where number of dimensions is greater than the number of samples.
    # Uses a subset of training points in the decision function (called support vectors), 
    # so it is also memory efficient.
    # Versatile: different Kernel functions can be specified for the decision function. 
    # Common kernels are provided, but it is also possible to specify custom kernels.

# The disadvantages of support vector machines include:
    # If the number of features is much greater than the number of samples, avoid over-fitting 
    # in choosing Kernel functions and regularization term is crucial.
    # SVMs do not directly provide probability estimates, these are calculated using an 
    # expensive five-fold cross-validation (see Scores and probabilities, below).

# class sklearn.svm.SVC(*, C=1.0, kernel='rbf', degree=3, gamma='scale', 
# coef0=0.0, shrinking=True, probability=False, tol=0.001, cache_size=200, 
# class_weight=None, verbose=False, max_iter=-1, 
# decision_function_shape='ovr', break_ties=False, random_state=None)

from sklearn import svm
import pandas as pd
import numpy as np

np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})

In [3]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Define the preprocessing pipeline
preprocessing_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),  # Impute missing values
    ('scaler', StandardScaler())  # Scale the features
])

##############################  TRAINING  ##############################################
# data
input_data = pd.read_csv('NHANES_data_stroke_train.csv')

#under sample the non-stroke
MI_positive = input_data[input_data['stroke'] == 1]
MI_negitive = input_data[input_data['stroke'] == 2].sample(frac=.03411675511751327)
input_data = pd.concat([MI_positive, MI_negitive])

# attributes
featurenames = ["Income","Sex","Age","Race","Edu","Diastolic","Systolic","Pulse","BMI","HDL","Trig","LDL","TCHOL","kidneys_eGFR","Diabetes","CurrentSmoker","isActive","isInsured"]
X = input_data[featurenames]
y = input_data['stroke']

# impute and scale the data
X = preprocessing_pipeline.fit_transform(X)

# split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.50, random_state=42)

# define and train the model
# kernals could be: ‘linear’, ‘poly’, ‘rbf’, ‘sigmoid’, ‘precomputed’.              
clf = svm.SVC(kernel="linear", C=1000, probability=True)
clf.fit(X_train, y_train)

# print accuracy info
print("accuracy train : ", clf.score(X_train, y_train))
print("accuracy test : ", clf.score(X_test, y_test))
print("predicted probabilities:\n", clf.predict_proba(X_test))

##############################  PREDICIOTN  ##############################################
# load data set
new_data = pd.read_csv("DEMO.csv")

# No stroke column so get rid of it
new_data = new_data.drop(columns=['stroke'])

# get attributes
X_new = new_data[featurenames]

# impute and scale the data
X_new = preprocessing_pipeline.fit_transform(X_new)

# Make predictions on the new data, run model
new_probabilities = clf.predict_proba(X_new)[:, 0]  # for output
new_predictions = clf.predict(X_new) # unsed, just for testing ratio of MI/noMI

# Get each sample's ID and write probabilities to the output CSV
new_participant_ids = new_data['SEQN']
new_output_df = pd.DataFrame({'SEQN': new_participant_ids, 'Pred_Probability': new_probabilities})
new_output_df.to_csv('SVMpred.csv', index=False)


accuracy train :  0.8222222222222222
accuracy test :  0.6888888888888889
predicted probabilities:
 [[0.735 0.265]
 [0.368 0.632]
 [0.277 0.723]
 [0.392 0.608]
 [0.376 0.624]
 [0.364 0.636]
 [0.170 0.830]
 [0.672 0.328]
 [0.430 0.570]
 [0.225 0.775]
 [0.642 0.358]
 [0.261 0.739]
 [0.447 0.553]
 [0.784 0.216]
 [0.274 0.726]
 [0.606 0.394]
 [0.293 0.707]
 [0.453 0.547]
 [0.442 0.558]
 [0.669 0.331]
 [0.178 0.822]
 [0.395 0.605]
 [0.725 0.275]
 [0.625 0.375]
 [0.500 0.500]
 [0.861 0.139]
 [0.585 0.415]
 [0.251 0.749]
 [0.778 0.222]
 [0.557 0.443]
 [0.685 0.315]
 [0.464 0.536]
 [0.710 0.290]
 [0.227 0.773]
 [0.184 0.816]
 [0.386 0.614]
 [0.819 0.181]
 [0.713 0.287]
 [0.510 0.490]
 [0.167 0.833]
 [0.769 0.231]
 [0.479 0.521]
 [0.300 0.700]
 [0.575 0.425]
 [0.601 0.399]
 [0.199 0.801]
 [0.575 0.425]
 [0.716 0.284]
 [0.784 0.216]
 [0.732 0.268]
 [0.094 0.906]
 [0.103 0.897]
 [0.745 0.255]
 [0.420 0.580]
 [0.644 0.356]
 [0.525 0.475]
 [0.567 0.433]
 [0.381 0.619]
 [0.157 0.843]
 [0.308 0.692]
 

In [4]:
#### See which data points are critical #####
# get the support vectors
print("clf support vectors: {}".format(clf.support_vectors_))
# get indices of support vectors
print("clf support vector indices: {}".format(clf.support_))
# get number of support vectors for each class
print("clf # of support vectors in each class: {}".format(clf.n_support_))

clf support vectors: [[-1.125 -1.015 1.225 ... 0.557 0.540 -0.348]
 [0.000 0.985 -0.495 ... -1.794 0.540 -0.348]
 [-0.943 0.985 -1.068 ... 0.557 0.540 -0.348]
 ...
 [0.074 0.985 0.733 ... 0.557 0.540 -0.348]
 [-0.155 0.985 -0.086 ... -1.794 0.540 -0.348]
 [-0.903 -1.015 0.570 ... 0.557 0.540 2.882]]
clf support vector indices: [  0  13  14  15  19  22  24  32  34  37  39  40  43  46  52  53  57  61
  62  63  64  67  74  81  82  85  88  89  93  97  99 100 110 112 113 115
 123 125 132 133 134   2   3   5   6  12  16  17  31  38  41  42  45  48
  51  58  59  60  66  73  75  76  77  87  92  94 103 106 108 109 118 119
 121 122 124 127 129]
clf # of support vectors in each class: [41 36]
